# Starting the project

Camera Calibration
Advanced Lane Finding Project
The goals / steps of this project are the following:

- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- Apply a distortion correction to raw images.
- Use color transforms, gradients, etc., to create a thresholded binary image.
- Apply a perspective transform to rectify binary image ("birds-eye view").
- Detect lane pixels and fit to find the lane boundary.
- Determine the curvature of the lane and vehicle position with respect to center.
- Warp the detected lane boundaries back onto the original image.
- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

### Calibrate camera using chessboard images, test on an image and show the results

In [ ]:
# Code Part 1
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle

%matplotlib qt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

#map corners of 2D image (image points) to the 3D coordinates of the real undistorted chess board corners (object points)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #write_name = 'corners_found'+str(idx)+'.jpg'
        #cv2.imwrite(write_name, img)
        #cv2.imshow('img', img)
        #cv2.waitKey(500)
        #plt.imshow(img) # will show the image in the jupiter notebook

#cv2.destroyAllWindows()
#save the file
### Calibrate camera, test on an image and show the results

# Test undistortion on an image
img = cv2.imread('camera_cal/calibration5.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('camera_cal/test_image_undistorted.jpg',dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ) )
#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

### Calibrate camera, test on an image and show the results

### Post utility

In [ ]:
# Code Part 2
#some common plot methods
import numpy as np
import cv2
import matplotlib.pyplot as plt

class post_utility():
    
    def __init__(self):
        self.name = "post_utility"
    
    
    def displayPlot(self, img, wid, hgt, wid_off, hgt_off, bottom_trim):
        #print('dislpay plot utility')
        # put the points on the image
        plt.imshow(img, cmap = 'gray')
        plt.plot(wid_off*0.95, hgt*bottom_trim, '.')   # bottom, left
        plt.plot(3*wid_off*1.1, hgt*bottom_trim, '.') # bottom, right
        plt.plot(0.55*wid, hgt_off, '.') # top, right
        plt.plot(0.45*wid, hgt_off, '.') #top, left

        #plt.imshow(warped, cmap = 'gray')
        plt.plot(wid_off*1.1, hgt, '.')   # bottom, left
        plt.plot(3*wid_off*0.9, hgt, '.') # bottom, right
        plt.plot(3*wid_off*0.9, 10, '.') # top, right
        plt.plot(wid_off*1.1, 10, '.') #top, left
        
        
    # code to put the markings on the saved image --- to be debugged later
    def displayPlotWarped(self, img, wid, hgt, wid_off, hgt_off, bottom_trim):
        #print('dislpay plot warped utility')
        # Create a named colour
        red = [0,0,255]
        blue = [255,0,0]
        binary_img[int(wid_off*0.95), int(hgt*bottom_trim)]=red
        binary_img[int(3*wid_off*1.1), int(hgt*bottom_trim)]=red
        binary_img[int(0.55*wid), int(hgt_off)]=red
        binary_img[int(0.45*wid), int(hgt_off)]=red

        warped[int(wid_off*0.95), int(hgt*bottom_trim)]=blue
        warped[int(3*wid_off*1.1), int(hgt*bottom_trim)]=blue
        warped[int(0.55*wid), int(hgt_off)]=blue
        warped[int(0.45*wid), int(hgt_off)]=blue

### Code for calculating gradient and color thresholds

In [ ]:
# Code Part 3
'''
#create the pipeline of combined grayscale (gradient, sobel, RGB), color scale (HLS) using a binary operation to detect 
#lane lines better than canny edge detection. For example this is more effient when shadows are present and lane line 
color changes near a bridge

for gray - use magnitude and direction gradient 
for color - use combination of R, S and H channels 

gray and R are good for right dotted line but not good if road color is white on the left side (on bridge)
R, S and H are good for yellow lane (on the left or tright of road but not good for dotted line)
'''
#need these imports to run as standalone
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import numpy as np
import cv2

# Calculate gradient magnitude and apply threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx ** 2 + sobely ** 2)  # Calculate the magnitude of the gradient
    scale_factor = np.max(gradmag) / 255  
    gradmag = (gradmag / scale_factor).astype(np.uint8)  # Scale to 8-bit (0 - 255) and convert to type = np.uint8  
    mag_binary = np.zeros_like(gradmag)  # apply a threshold
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1   # Create binary mask where mag thresholds are met
    return mag_binary  # Return this mask as your binary_output image


# Calculate gradient direction and apply threshold
def dir_thresh(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    # Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    absgraddir = np.arctan2(abs_sobely, abs_sobelx)
    # Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return dir_binary  # Return this mask as your binary_output image


# Define a function that thresholds the R-channel of RGB
def rgb_r_thresh(img, thresh=(200, 255)):
    red_ch = img[:, :, 0]
    #plt.imshow(red_ch, cmap='gray')
    binary_output = np.zeros_like(red_ch)
    binary_output[(red_ch > thresh[0]) & (red_ch <= thresh[1])] = 1
    return binary_output


# Define a function that thresholds the R-channel of RGB
def rgb_g_thresh(img, thresh=(200, 255)): #thresh=(200, 255)
    green_ch = img[:, :, 1]
    #plt.imshow(green_ch, cmap='gray')
    binary_output = np.zeros_like(green_ch)
    binary_output[(green_ch > thresh[0]) & (green_ch <= thresh[1])] = 1
    return binary_output

# Define a function that thresholds the S-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_s_thresh(img, thresh=(90, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)   # Convert to HLS color space
    s_ch = hls[:, :, 2]  # Extract S channel
    binary_output = np.zeros_like(s_ch)
    binary_output[(s_ch > thresh[0]) & (s_ch <= thresh[1])] = 1  # Apply a threshold to the S channel
    return binary_output  # Return a binary image of threshold result


# Define a function that thresholds the L-channel of HLS
# Use exclusive lower bound (>) and inclusive upper (<=)
def hls_l_thresh(img, thresh=(90, 255)):
    #print('img:', img)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)  # Convert to HLS color space
    l_ch = hls[:, :, 1]  # Extract S channel
    #plt.imshow(l_ch, cmap='gray')
    #print('l_ch:',l_ch)
    binary_output = np.zeros_like(l_ch)
    binary_output[(l_ch > thresh[0]) & (l_ch <= thresh[1])] = 1  # Apply a threshold to the S channel
    return binary_output  # Return a binary image of threshold result


# Define a function that thresholds the H-channel of HLS
def hls_h_thresh(img, thresh=(15, 100)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)  # Convert to HLS color space
    h_ch = hls[:, :, 0] # Extract S channel
    binary_output = np.zeros_like(h_ch)
    binary_output[(h_ch > thresh[0]) & (h_ch <= thresh[1])] = 1  # Apply a threshold to the S channel
    return binary_output  # Return a binary image of threshold result

def hsv_v_thresh(img, thresh=(180, 255)):
        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)  # Convert to HLS color space
        v_ch = hsv[:, :, 2]  # Extract S channel
        binary_output = np.zeros_like(v_ch)
        binary_output[(v_ch > thresh[0]) & (v_ch <= thresh[1])] = 1  # Apply a threshold to the V channel
        return binary_output  # Return a binary image of threshold result

# Sliding window using convolution

In [ ]:
# Code Part 4
#------------
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2

class sliding_window_conv():
    
    lane_start_points = []
    recent_centers = []
    def __init__(self, window_width_in, window_height_in, margin_in, ncenters_in, xm_per_pix_in, ym_per_pix_in):
        self.name = "sliding_win_conv"
        # window settings
        self.window_width = window_width_in  # search window width  # 50 
        self.window_height = window_height_in  # 80 # Break image into 9 vertical layers since image height is 720
        self.margin = margin_in # 100 # How much to slide left and right for searching
    
        #self.recent_centers = []
        #self.lane_start_points = []
        #m_per_pixel_x  # meters per pixel in x-axis
        #m_per_pixel_y  # meters per pixel in y-axis
        self.smoothing_factor = ncenters_in  #10  so the the curve does not jump around
        # Define conversions in x and y from pixels space to meters
        self.xm_per_pix = xm_per_pix_in  #3.7/700 # meters per pixel in x dimension
        self.ym_per_pix = ym_per_pix_in  #30/720 # meters per pixel in y dimension
        

    def window_mask(self, width, height, img_ref, center,level):
        output = np.zeros_like(img_ref)
        output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
        return output

    
    #if not white pixcel is found, windows are moving to left, should stay at previous location itself
    # otherwise for next iteration, it will move away
    # also can check if its equidistant from left lane -- in case the left lane pixels match the window, use it as reference
    #input is a warped image that has been set to perspective mode
    def find_window_centroids(self, image): #, window_width, window_height, margin

        window_width = self.window_width
        window_height = self.window_height
        margin = self.margin
        
        window_centroids = [] # Store the (left,right) window centroid positions per level
        # start update - store recent values for conv_signal of left and right window, initialize with 50
        offset_temp = window_width/2
        avgVal = int((np.abs(offset_temp-margin) + offset_temp+margin)/2)
        #print('offset_temp/margin/avgVal: ', offset_temp, '/', margin, '/', avgVal)
        window_lconv = [avgVal] # avgVal = 50 for current settings
        window_rconv = [avgVal] # same for right window
        ##print('window_lconv: ', window_lconv, ' window_rconv: ', window_rconv)
        # end update to initialize the lconv and rconv values
        window = np.ones(window_width) # Create our window template that we will use for convolutions

        # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
        # and then np.convolve the vertical image slice with the window template 

        # Sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
        # Add what we found for the first layer
        window_centroids.append((l_center,r_center))
        #---------------------------------------------------
        # minimize lane starting point drift between frames
        # if this works, may be we can skip some frames but not sure how curvature at farther end is impacted
        #find average start points for the lanes
        self.lane_start_points.append((l_center,r_center))
        avg_start_points = np.average(self.lane_start_points[-20:], axis = 0) # 10
        #print('size of lanepoints, avg_start_points: ', len(self.lane_start_points), '/',avg_start_points)
        #print('1 l_center: ', l_center, ' r_center: ', r_center)
        #if start points are 25 points away from average set to average
        if(abs(l_center - avg_start_points[0]) >= 35): #25
            l_center = avg_start_points[0]
            
        if(abs(r_center - avg_start_points[1]) >= 35): #25
            r_center = avg_start_points[1]
        
        #print('2 l_center: ', l_center, ' r_center: ', r_center)
        #print('avg start points 1: ',avg1)
        #print('window_centroids 1: ', window_centroids)
        #---------------------------------------------------
        
        # Go through each layer looking for max pixel locations
        for level in range(1,(int)(image.shape[0]/window_height)):
            # convolve the window into the vertical slice of the image
            image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
            conv_signal = np.convolve(window, image_layer)
            # Find the best left centroid by using past left center as a reference
            # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
            offset = window_width/2
            l_min_index = int(max(l_center+offset-margin,0))
            l_max_index = int(min(l_center+offset+margin,image.shape[1]))
            lconv = np.argmax(conv_signal[l_min_index:l_max_index])
            ##print('lconv 1: ', lconv)
            if(lconv == 0):
                 lconv = int(np.average(window_lconv))
                    
            ##print('lconv 2: ', lconv)
            l_center = lconv+l_min_index-offset
            # Find the best right centroid by using past right center as a reference
            #print('r_center+offset-margin: ', r_center, '/',offset, '/',margin)
            r_min_index = int(max(r_center+offset-margin,0))
            r_max_index = int(min(r_center+offset+margin,image.shape[1]))
            #print('len-conv_signal, r_min_index, r_max_index: ', len(conv_signal),'/', r_min_index,'/', r_max_index)
            #print('conv_signal[r_min_index:r_max_index]: ', conv_signal[r_min_index:r_max_index])
            rconv = np.argmax(conv_signal[r_min_index:r_max_index])
            #print('rconv 1: ', rconv)
            if(rconv == 0):
                 rconv = int(np.average(window_rconv))
                    
            #print('rconv 2: ', rconv)
            r_center = rconv+r_min_index-offset
            
            #code to fix  when a 0 signal was found for convolution
            if(lconv != 0):
                window_lconv.append(lconv)
            if(rconv != 0):
                window_rconv.append(rconv)
                
            ##print('avg_lconv: ', int(np.average(window_lconv)), ' avg_rconv: ', int(np.average(window_rconv)))
            #---------------------
            # Add what we found for that layer
            window_centroids.append((l_center,r_center))

            
        self.recent_centers.append(window_centroids)
        #print('window_centroids 2: ', window_centroids)
        #print('np.average(self.recent_centers[-self.smoothing_factor:], axis=0): ', np.average(self.recent_centers[-self.smoothing_factor:], axis=0))
        ##print('len(window_centroids): ', len(window_centroids))
        ##print('len(np.avg.self.recCen): ', len(np.average(self.recent_centers[-self.smoothing_factor:], axis=0)))
        
        return np.average(self.recent_centers[-self.smoothing_factor:], axis=0)
        #return window_centroids



# List of steps performed in advanced_lane_find 
### Apply a distortion correction to raw images.
### Use color transforms, gradients, etc., to create a thresholded binary image.
### Detect lane pixels and fit to find the lane boundary
### Determine the curvature of the lane and vehicle position with respect to center (curvature uses polyfit through lane pixels and then find curvature)
### Warp the detected lane boundaries back onto the original image.
### Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# Code Part 5
#------------
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle

#def advanced_lane_find(img):
class advanced_lane_find():
    
    def __init__(self):
        self.name = "sliding_win_conv"
        self.leftx_val = []   # to save left poly points
        self.rightx_val = []
        self.smoothing_factor_poly = 32 #8 # 12 is also good # average poly coefs for of 32 frames
        self.l_points_val = []   # to save left right lane points
        self.r_points_val = []
        
    
    def compute_lane(self, img):
        # test the calibration and distortion correction on the test_images 
        # Make a list of calibration images
        #images = glob.glob('test_images/test*.jpg')

        #read the camera calibration data from the pickle file
        camera_cal_pickle_file = "camera_cal/wide_dist_pickle.p"
        infile = open(camera_cal_pickle_file,'rb')
        new_dist_pickle = pickle.load(infile)
        mtx = new_dist_pickle["mtx"]
        dist = new_dist_pickle["dist"]
        infile.close()
        img_3d = img

        #read the mtx, dist from pickle -- to be added
        undistort = cv2.undistort(img, mtx, dist, None, mtx) #undistorted image
        #test for distortion correction
        img = undistort
        ksize = 3 # Choose a larger odd number to smooth gradient measurements
        mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100)) #mag_thresh=(0, 255)
        ksize = 15
        dir_binary = dir_thresh(img, sobel_kernel=ksize, thresh=(0.7, 1.2)) #15, thresh=(0.7, 1.3) 0, np.pi/2

        #rgb_r_binary = rgb_r_thresh(img, thresh=(200, 255)) #200, 255
        #rgb_g_binary = rgb_g_thresh(img, thresh=(200,255)) #200, 255

        hls_h_binary = hls_h_thresh(img, thresh=(12, 80)) #15, 100
        hls_l_binary = hls_l_thresh(img, thresh=(200, 255)) #15, 100
        hls_s_binary = hls_s_thresh(img, thresh=(100, 255)) #90, 255)

        hsv_v_binary = hsv_v_thresh(img, thresh=(180, 255))  # 90, 255)
        # Combine the two or more binary thresholds
        combined_binary = np.zeros_like(dir_binary)    
        #combined_binary[( ((rgb_r_binary == 1) | (rgb_g_binary == 1)) |
        #           (((hls_h_binary == 1) | (hls_s_binary == 1)) & (dir_binary == 1) )
        #          )] = 1

        combined_binary[ ( (mag_binary == 1) | (((hls_h_binary == 1)|(hls_s_binary == 1) ) ) ) & (hsv_v_binary == 1)] = 1

        #plt.imshow(combined_binary, cmap='gray')
        #print("./test_images/binary_"+fname[12:])

        binary_img = combined_binary.astype('uint8') * 255 
        #cv2.imwrite('./test_images/binary_'+fname[12:], binary_img)  #combined_binary.astype('uint8') * 255

        img = binary_img
        #create the perspective transform of lanes lines
        wid = img.shape[1]     # 1280
        hgt = img.shape[0]     # 720
        img_size = (wid, hgt)  # img_size = (gray.shape[1], gray.shape[0])  # Grab the image shape

        wid_off = 0.25*wid   # width offset
        hgt_off = 0.65*hgt #0.64*hgt     # height offset
        bottom_trim = .95 #.96  .94 -- can be more as dashboard is curved
        #0.6*wid = wid_off * 2.4;  0.4*wid= wid_off * 1.6
        # source points 
        #4;3  -- order of points
        #1;2
        src = np.float32([ [wid_off*0.95, hgt*bottom_trim], [3*wid_off*1.1, hgt*bottom_trim], 
                           [0.55*wid, hgt_off ], [0.45*wid, hgt_off] ])
        # destination points
        #dst = np.float32([ [wid_off,hgt], [3*wid_off, hgt], [3*wid_off, 0], [wid_off, 0]])
        dst = np.float32([ [wid_off*1.1,hgt], [3*wid_off*0.9, hgt], [3*wid_off*0.9, 0], [wid_off*1.1, 0]])

        M = cv2.getPerspectiveTransform(src, dst)  # to create parallel lanes from original road
        Minv = cv2.getPerspectiveTransform(dst, src) # to recreate perspective lanes

        warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

        cv2.imwrite('./test_images/binary_'+fname[12:], binary_img)  #combined_binary.astype('uint8') * 255
        cv2.imwrite('./test_images/pers_'+fname[12:], warped)
        #print('filname: ', './test_images/pers_'+fname[12:])

        #for documentation only
        #tempPost = post_utility()
        #tempPost.displayPlot(img, wid, hgt, wid_off, hgt_off, bottom_trim)
        #------------------------------
        # first create the sliding windows using convolution
        # fit a polynomial through it and create the overlaid pictures
        window_width = 50  # 25 
        window_height = 80 # 60 #80
        margin = 50 #25

        # Read in a thresholded image
        #warped = mpimg.imread('warped_example.jpg')
        ncenters = 36 #18  #36#54 #18  #24#48 #15 
        #  window_height=60, each frame 12 centers, average of 4 frames -> ncenters=48
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        ym_per_pix = 30/720 # meters per pixel in y dimension

        window_conv = sliding_window_conv(window_width, window_height, margin, ncenters, xm_per_pix, ym_per_pix)
        window_centroids = window_conv.find_window_centroids(warped)
        myTemplate = np.zeros_like(warped) #to use for poly fill between lanes

        ### Detect lane pixels and fit to find the lane boundary.
        #this is for histogram to find location of lane pixels/lines in the image
        # If we found any window centers
        if len(window_centroids) > 0:
            # Points used to draw all the left and right windows
            l_points = np.zeros_like(warped)
            r_points = np.zeros_like(warped)
            leftx = []
            rightx = []
            # Go through each level and draw the windows 	
            for level in range(0,len(window_centroids)):
                # Window_mask is a function to draw window areas
                l_mask = window_conv.window_mask(window_width,window_height,warped,window_centroids[level][0],level)
                r_mask = window_conv.window_mask(window_width,window_height,warped,window_centroids[level][1],level)
                # Add graphic points from window mask here to total pixels found 
                l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
                r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
                leftx.append(window_centroids[level][0]) #used for polyfit
                rightx.append(window_centroids[level][1]) #used for polyfit

            #----------------------------------------------------------
            #add leftx, rightx vals to class global variable to average
            self.leftx_val.append(leftx)
            self.rightx_val.append(rightx)
            #just store 35 frame data
            self.leftx_val = self.leftx_val[-35:]
            self.rightx_val = self.rightx_val[-35:]
            
            #print('len self.leftx, leftx: ', len(self.leftx_val), len(leftx) )
            #print('len self.rightx, rightx: ', len(self.rightx_val), len(rightx))
            #print(' self.leftx_val, leftx: ', self.leftx_val, leftx )
            # to save left right lane points
            #self.l_points_val.append(l_points) 
            #self.r_points_val.append(r_points)
            
            #----------------Not needed for project, for understanding only-------------------
            #----------------------------------------------------------
            # Draw the results
            template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
            #--------------------------------------------------
            init_template = template # to save the polyfill
            ##pointsToAvg = self.smoothing_factor_poly * npoints
            ##get average of the points from smoothing_factor_poly(2) frames for the polynomial coefficients to smooth the lines
            #pointsToAvg = 2 * npoints
            #temp_l_points =  np.average(self.l_points_val[(-pointsToAvg):], axis=0)
            #temp_r_points =  np.average(self.r_points_val[(-pointsToAvg):], axis=0)
            #
            #-----------------------------------------------------
            zero_channel = np.zeros_like(template) # create a zero color channel
            template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
            warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
            #prints the windows that mark the lanes -- not used in video result
            output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

            # start code to fill the area between the lanes
            #myTemplate = np.zeros_like(warped)
            #print('template: ', template)
            #print('template.shape: ', template.shape[0]) #720, 1280
            rows = template.shape[0] #height
            col = template.shape[1]  #width
            check = 255
            for row in range(rows): #720
                try:
                    templist = init_template[row].tolist()
                    firstval = templist.index(check)
                    secondval = len(templist) - 1 - templist[::-1].index(check)
                    myTemplate[row][firstval:secondval] = 255
                except ValueError:
                    print('value is not in list..')
            #.............
            myTemplate = np.array(cv2.merge((zero_channel,myTemplate,zero_channel)),np.uint8)
            #This output image prints the lanes marked with windows with polyfill in between, not used in video result
            output = cv2.addWeighted(output, 1, myTemplate, 0.5, 0.0)
            #cv2.imwrite('./test_images/output_conv_'+fname[12:], output)
            #conv_warped = cv2.addWeighted(warpage, 1, myTemplate, 0.4, 0.0) # overlay the orignal road image with window results
            #end code to fill area between the lanes
            #----------------Not needed for project, for understanding only-------------------
        else:
            # If no window centers found, just display orginal road image
            output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

        # write the convolution results
        cv2.imwrite('./test_images/conv_'+fname[12:], output)

        # assign warped to conv_warped, so that the area between the lanes is highlighted
        #warped = conv_warped
        #create the polyfit
        # this has to be understood and updated
        warp_hgt = warped.shape[0]
        yvals = range(0, warp_hgt)
        ploty = yvals
        res_yvals = np.arange(warp_hgt - (window_height/2), 0, -window_height)  # can also use linspace which also returns an array

        npoints = len(leftx) #same as img.shape(0)/window_heigth = 720/80 = 9
        #print('leftx: ', len(leftx), ' : ',leftx)
        #print('rightx: ', len(rightx), ' : ',rightx)
        #get average of the points from smoothing_factor_poly(2) frames for the polynomial coefficients to smooth the lines
        #pointsToAvg = self.smoothing_factor_poly * npoints  #8*9 =72 points -- 72 polynomials are averaged = 72 frames @ 1 poly/frame
        pointsToAvg = self.smoothing_factor_poly #*4  # average 32 frames
        temp_leftx =  np.average(self.leftx_val[(-pointsToAvg):], axis=0)
        temp_rightx =  np.average(self.rightx_val[(-pointsToAvg):], axis=0)
        #print('len, tempLeftx : ', len(temp_leftx), ' :\n ',temp_leftx)
        #print('self.leftx_val[(-pointsToAvg):]: ', self.leftx_val[(-pointsToAvg):])
        #-----------------------------------------------------------------------------
        #/////////////////////////////////////////////////////////////////
        #recalculate the polyfill between lanes using the average leftx, rightx values
        #to make sure the fill is between the new lines, otherwise it will show out of lanes
        
        window_centroids_avg = np.array((temp_leftx, temp_rightx)).T
        myTemplateAvg = np.zeros_like(warped) #to use for poly fill between lanes
        if len(window_centroids_avg) > 0:
            # Points used to draw all the left and right windows
            l_points = np.zeros_like(warped)
            r_points = np.zeros_like(warped)
            #leftx = [] #may not be needed here
            #rightx = []  #may not be needed here
            # Go through each level and draw the windows 	
            for level in range(0,len(window_centroids_avg)):
                # Window_mask is a function to draw window areas
                l_mask = window_conv.window_mask(window_width,window_height,warped,window_centroids_avg[level][0],level)
                r_mask = window_conv.window_mask(window_width,window_height,warped,window_centroids_avg[level][1],level)
                # Add graphic points from window mask here to total pixels found 
                l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
                r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
                #leftx.append(window_centroids_avg[level][0]) #used for polyfit
                #rightx.append(window_centroids_avg[level][1]) #used for polyfit

            #----------------------------------------------------------
            #----------------Not needed for project, for understanding only-------------------
            # Draw the results
            templateAvg = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
            init_templateAvg = templateAvg
            #-----------------------------------------------------
            zero_channel = np.zeros_like(templateAvg) # create a zero color channel
            templateAvg = np.array(cv2.merge((zero_channel,templateAvg,zero_channel)),np.uint8) # make window pixels green
            warpageAvg = np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
            #prints the windows that mark the lanes -- not used in video result
            outputAvg = cv2.addWeighted(warpageAvg, 1, templateAvg, 0.5, 0.0) # overlay the orignal road image with window results

            # start code to fill the area between the lanes
            rows = templateAvg.shape[0] #height
            col = templateAvg.shape[1]  #width
            check = 255
            for row in range(rows): #720
                try: 
                    templist = init_templateAvg[row].tolist()
                    firstval = templist.index(check)
                    secondval = len(templist) - 1 - templist[::-1].index(check)
                    myTemplateAvg[row][firstval:secondval] = 255
                except ValueError:
                    print('value is not in list..Avg')
            #.............
            myTemplateAvg = np.array(cv2.merge((zero_channel,myTemplateAvg,zero_channel)),np.uint8)
            #This output image prints the lanes marked with windows with polyfill in between, not used in video result
            outputAvg = cv2.addWeighted(outputAvg, 1, myTemplate, 0.5, 0.0)
            #cv2.imwrite('./test_images/output_conv_'+fname[12:], output)
            #conv_warpedAvg = cv2.addWeighted(warpage, 1, myTemplateAvg, 0.4, 0.0) # overlay the orignal road image with window results
            #end code to fill area between the lanes
            #----------------Not needed for project, for understanding only-------------------     
        #end recalculate the polyfill
        #///////////////////////////////////////////
        #-----------------------------------------------------------------------
        leftx = temp_leftx
        rightx = temp_rightx
        ###  Fit a second order polynomial to each using `np.polyfit` ###
        left_fit = np.polyfit(res_yvals, leftx, 2)  # lefty, leftx
        right_fit = np.polyfit(res_yvals, rightx, 2)  # righty, rightx

        try:
            left_fitx = left_fit[0] * np.power(ploty, 2) + left_fit[1] * ploty + left_fit[2]
            right_fitx = right_fit[0] * np.power(ploty, 2) + right_fit[1] * ploty + right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1 * np.power(ploty, 2) + 1 * ploty
            right_fitx = 1 * np.power(ploty, 2) + 1 * ploty

        left_fitx = np.array(left_fitx, np.int32) 
        right_fitx = np.array(right_fitx, np.int32) 

        # this has to updated later
        left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2, left_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
        right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
        middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)


        ### Warp the detected lane boundaries back onto the original image.
        ### Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
        out_img = np.dstack((warped, warped, warped)) * 255
        #create red and blue lane lines
        road = np.zeros_like(out_img)  # this is a blank image
        cv2.fillPoly(road, np.int_([left_lane]), (255, 0, 0))
        cv2.fillPoly(road, np.int_([right_lane]), (0, 0, 255))
        #print('road: ', road)
        #create a background white line so that the road lanes can be highlighted
        road_bkg = np.zeros_like(out_img)  # actual image
        cv2.fillPoly(road_bkg, [left_lane], (255, 255, 255))
        cv2.fillPoly(road_bkg, [right_lane], (255, 255, 255))

        road_warped = cv2.warpPerspective(road, Minv, img_size, flags=cv2.INTER_LINEAR)
        road_warped_bkg = cv2.warpPerspective(road_bkg, Minv, img_size, flags=cv2.INTER_LINEAR)
        #--------------
        #to add green poly fill, use data from myTemplate
        #lane_warped = cv2.warpPerspective(myTemplate, Minv, img_size, flags=cv2.INTER_LINEAR) #warp lane
        lane_warped = cv2.warpPerspective(myTemplateAvg, Minv, img_size, flags=cv2.INTER_LINEAR) #warp lane
        final_result = cv2.addWeighted(img_3d, 1, lane_warped, 0.4, 0.0) # overlay the orignal road image with window results
        img_3d = final_result
        #--------------
        base = cv2.addWeighted(img_3d, 1.0, road_warped_bkg, -1.0,0.0)
        result = cv2.addWeighted(base, 1.0, road_warped, 1.0,0.0)
        #cv2.imwrite('./test_images/polyfill_'+fname[12:], result)

        #----------------------------------------------------
        # check how the region of interest marked by source points is performing
        # create the polyfill of the initial source points box, No need to warp this
        #init_box = np.zeros_like(out_img)  # actual image
        #cv2.fillPoly(init_box, np.int_([src]), (255, 153, 51))  #$53, 255, 51
        #result = cv2.addWeighted(result, 1.0, init_box, 0.6,0.0)
        #-----------------------------------------------------
    
        #-------------------------------------------------------------------------------------
        ### Determine the curvature of the lane and vehicle position with respect to center.
        #curvature uses polyfit through lane pixels and then find curvature
        # text on the image and video    
        ploty = np.array(res_yvals, np.int32)
        leftx_f = np.array(leftx, np.int32)
        rightx_f = np.array(rightx, np.int32)
        y_eval = np.max(ploty)

        left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx_f*xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx_f*xm_per_pix, 2)

        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

        #calculate offset of the car on the road
        camera_center = (left_fitx[-1] + right_fitx[-1])/2
        center_diff = (camera_center - warped.shape[1]/2) * xm_per_pix
        vehicle_pos = 'left'
        if center_diff <= 0:
            vehicle_pos = 'right'

        cv2.putText(result, 'Radius of Curvature = '+ str(round(right_curverad, 3))+'(m)', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
        cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff, 3)))+'(m) '+vehicle_pos+' of center', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
        cv2.imwrite('./test_images/final_'+fname[12:], result)

        return result
        #----------------------------------------------------------
        # Colors in the left and right lane regions
        #out_img[lefty, leftx] = [255, 0, 0]  # color left lane pixels -- this info is missing in convolution, get from window code
        #out_img[righty, rightx] = [0, 0, 255] # color right lane pixels




In [ ]:
# Code Part 6
#------------
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# Code Part 7
#------------
output_video = "test_videos/submit_video.mp4"
input_video = "test_videos/project_video.mp4" 
#clip2 = VideoFileClip('test_videos/project_video.mp4').subclip(0,3)
clip1 = VideoFileClip("test_videos/project_video.mp4")#.subclip(0,10) #remove subclip after testing
#0,10 0,35
#15,42 is the window of error
#25, 32 is the narrower window to check lane and car
temp_lane = advanced_lane_find()
output_clip = clip1.fl_image(temp_lane.compute_lane)
#output_clip.write_videofile(output_video, audio=False)
%time output_clip.write_videofile(output_video, audio=False)

In [ ]:
# Code Part 8 - visualize project video
#--------------------------------------
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_video))

In [ ]:
# Code Part 9
#------------
output_challenge_video = "test_videos/submit_challenge_video.mp4"
input_challenge_video = "challenge_video.mp4" 
clip3 = VideoFileClip(input_challenge_video) #.subclip(0,4) #remove subclip after testing
temp_lane = advanced_lane_find()
output_ch_clip = clip3.fl_image(temp_lane.compute_lane)
%time output_ch_clip.write_videofile(output_challenge_video, audio=False)

In [ ]:
# Code Part 10
#-------------
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_challenge_video))

In [ ]:
# Code Part 11
#------------
input_challenge_video = "harder_challenge_video.mp4" 
output_hard_challenge_video = "test_videos/submit_hard_challenge_video.mp4"
clip3 = VideoFileClip(input_challenge_video) #.subclip(0,5) #remove subclip after testing
temp_lane = advanced_lane_find()
output_ch_clip = clip3.fl_image(temp_lane.compute_lane)
%time output_ch_clip.write_videofile(output_hard_challenge_video, audio=False)

In [ ]:
# Code Part 12
#-------------
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_hard_challenge_video))

### BELOW IS FOR TESTING MODULES  
#### This is not part of project code
#### Apply a perspective transform to rectify binary image ("birds-eye view").

In [ ]:
# USED FOR TESTING ONLY
#----------------------

In [ ]:
# USED FOR TESTING ONLY
#----------------------
#mask region of interest -- needs debugging
 # draw region of interest
    # Define a triangle region of interest 
    # The origin (x=0, y=0) is in the upper left in image processing
    #left_bottom = [0, 539]
    #right_bottom = [900, 300]
    #apex = [400, 0]
    # source points 
    #4;3  -- order of points
    #1;2
    # Fit lines (y=Ax+B) to identify the  3 sided region of interest
    # np.polyfit() returns the coefficients [A, B] of the fit
    
    
    fit_left = np.polyfit((src[0][0], src[3][0]), (src[0][1], src[3][1]), 1) # (left_bottom[0], apex[0]), (left_bottom[1], apex[1])
    fit_right = np.polyfit((src[1][0], src[2][0]), (src[1][1], src[2][1]), 1) #(right_bottom[0], apex[0]), (right_bottom[1], apex[1])
    fit_bottom = np.polyfit((src[0][0], src[1][0]), (src[0][1], src[1][1]), 1) #(left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1])
    fit_top = np.polyfit((src[2][0], src[3][0]), (src[2][1], src[3][1]), 1)
    
    # Find the region inside the lines
    XX, YY = np.meshgrid(np.arange(0, wid), np.arange(0, hgt))
    region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                        (YY > (XX*fit_right[0] + fit_right[1])) & \
                        (YY < (XX*fit_bottom[0] + fit_bottom[1])) & \
                        (YY < (XX*fit_top[0] + fit_top[1]))
                         
    print('XX: ', XX)
    print('YY: ', YY)
    print('region_thresholds: ', region_thresholds)
    
    region_select = np.copy(img)
    # Color pixels red which are inside the region of interest
    region_select[region_thresholds] = [255, 0, 0]

    # Display the image
    plt.imshow(region_select)




### Detect lane pixels and fit to find the lane boundary.

In [ ]:
this is for histogram to find location of lane pixels/lines in the image

### Determine the curvature of the lane and vehicle position with respect to center.

In [ ]:
curvature uses polyfit through lane pixels and then find curvature


# Define a class to receive the characteristics of each line detection

In [ ]:
# Define a class to receive the characteristics of each line detection
# Was not able to implement this in the time provided
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  
        

In [ ]:
### TEST ONLY # For testing the code

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
    
# to be updated 0
# idea to create a single channel for r,g,b = [240,240,240], merge to get green
def fillSingleChannel(img):
    myTemplate = np.zeros_like(warped)
    #print('template: ', template)
    #print('template.shape: ', template.shape[0]) #720, 1280
    #print('template[0]: ', template[0].max()) #720, 1280
    #print('r_points: ', r_points) #720, 1280
    rows = template.shape[0] #height
    col = template.shape[1]  #width
    check = 255
    for row in range(rows): #720
        templist = template[row].tolist()
        firstval = templist.index(check)
        secondval = len(templist) - 1 - templist[::-1].index(check)
        myTemplate[row][firstval:secondval] = 255
    #print(myTemplate)
    #.............
        
# test the calibration and distortion correction on the test_images 
# Make a list of calibration images
images = glob.glob('test_images/test*.jpg')

#read the camera calibration data from the pickle file
camera_cal_pickle_file = "camera_cal/wide_dist_pickle.p"
infile = open(camera_cal_pickle_file,'rb')
new_dist_pickle = pickle.load(infile)
mtx = new_dist_pickle["mtx"]
dist = new_dist_pickle["dist"]
infile.close()

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img_3d = cv2.imread(fname)
    
    img = img_3d
    #read the mtx, dist from pickle -- to be added
    undistort = cv2.undistort(img, mtx, dist, None, mtx) #undistorted image
    #test for distortion correction
    #cv2.imwrite('./test_images/undist_'+fname[12:],dst)
    #plt.imshow(dst)
        
    #print('dst: ',dst)
    #Try different combinations and see what you get.
    #For example, here is a selection for pixels where both the xx and yy gradients meet the threshold criteria,
    #or the gradient magnitude and direction are both within their threshold values.  
    img = undistort
    ksize = 3 # Choose a larger odd number to smooth gradient measurements
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100)) #mag_thresh=(0, 255)
    ksize = 15
    dir_binary = dir_thresh(img, sobel_kernel=ksize, thresh=(0.7, 1.2)) #15, thresh=(0.7, 1.3) 0, np.pi/2
    
    rgb_r_binary = rgb_r_thresh(img, thresh=(200, 255)) #200, 255
    rgb_g_binary = rgb_g_thresh(img, thresh=(200,255)) #200, 255

    hls_h_binary = hls_h_thresh(img, thresh=(12, 80)) #15, 100
    hls_l_binary = hls_l_thresh(img, thresh=(200, 255)) #15, 100
    hls_s_binary = hls_s_thresh(img, thresh=(100, 255)) #90, 255)

    # Combine the two or more binary thresholds
    combined_binary = np.zeros_like(dir_binary)
    #combined[ ((mag_binary == 1) & (dir_binary == 1)) | ((rgb_r_binary == 1) & (hls_h_binary == 1) & (hls_s_binary == 1)) ] = 1
  
    #combined_binary[ (((mag_binary == 1) & ((rgb_r_binary == 1) | (rgb_g_binary == 1)) ) |
    #      ((hls_h_binary == 1) & (hls_s_binary == 1)))  ] = 1

    #& (dir_binary == 1) # may be add this for challenge video
    #combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    combined_binary[( ((rgb_r_binary == 1) | (rgb_g_binary == 1)) |
               (((hls_h_binary == 1) | (hls_s_binary == 1)) & (dir_binary == 1) )
              )] = 1
    
    #combined_binary = mag_binary
    plt.imshow(combined_binary, cmap='gray')
    print("./test_images/binary_"+fname[12:])
        
    binary_img = combined_binary.astype('uint8') * 255 
    #cv2.imwrite('./test_images/binary_'+fname[12:], binary_img)  #combined_binary.astype('uint8') * 255
    
    img = binary_img
    #create the perspective transform of lanes lines
    wid = img.shape[1]     # 1280
    hgt = img.shape[0]     # 720
    img_size = (wid, hgt)  # img_size = (gray.shape[1], gray.shape[0])  # Grab the image shape

    wid_off = 0.25*wid   # width offset
    hgt_off = 0.64*hgt     # height offset
    bottom_trim = .96 #.94 -- can be more as dashboard is curved
    #0.6*wid = wid_off * 2.4;  0.4*wid= wid_off * 1.6
    # source points 
    #4;3  -- order of points
    #1;2
    src = np.float32([ [wid_off*0.95, hgt*bottom_trim], [3*wid_off*1.1, hgt*bottom_trim], 
                       [0.55*wid, hgt_off ], [0.45*wid, hgt_off] ])
    # destination points
    #dst = np.float32([ [wid_off,hgt], [3*wid_off, hgt], [3*wid_off, 0], [wid_off, 0]])
    dst = np.float32([ [wid_off*1.1,hgt], [3*wid_off*0.9, hgt], [3*wid_off*0.9, 0], [wid_off*1.1, 0]])
    
    #print('src: ', src)
    #print('dst: ', dst)
    M = cv2.getPerspectiveTransform(src, dst)  # to create parallel lanes from original road
    Minv = cv2.getPerspectiveTransform(dst, src) # to recreate perspective lanes

    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    cv2.imwrite('./test_images/binary_'+fname[12:], binary_img)  #combined_binary.astype('uint8') * 255
    cv2.imwrite('./test_images/pers_'+fname[12:], warped)
    print('filname: ', './test_images/pers_'+fname[12:])
    
    #for documentation only
    tempPost = post_utility()
    tempPost.displayPlot(img, wid, hgt, wid_off, hgt_off, bottom_trim)
    #cv2.imwrite('./test_images/pointsForPers_'+fname[12:], warped)
    #------------------------------
    # first create the sliding windows using convolution
    # fit a polynomial through it
    # create the overlaid pictures
    
    window_width = 50 #25 
    window_height = 80  #60 #80
    margin = 50  #25
    
    # Read in a thresholded image
    #warped = mpimg.imread('warped_example.jpg')
    ncenters = 8 # 15
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    ym_per_pix = 30/720 # meters per pixel in y dimension
        
    window_conv = sliding_window_conv(window_width, window_height, margin, ncenters, xm_per_pix, ym_per_pix)
    window_centroids = window_conv.find_window_centroids(warped)
    myTemplate = np.zeros_like(warped) #to use for poly fill between lanes
    print('len(window_centroids): ',len(window_centroids))
    # If we found any window centers
    if len(window_centroids) > 0:

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)
        leftx = []
        rightx = []
        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_conv.window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_conv.window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
            leftx.append(window_centroids[level][0]) #used for polyfit
            rightx.append(window_centroids[level][1]) #used for polyfit
            
        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        init_template = template # to save the polyfill
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

        # start code to fill the area between the lanes
        #myTemplate = np.zeros_like(warped)
        #print('template: ', template)
        #print('template.shape: ', template.shape[0]) #720, 1280
        rows = init_template.shape[0] #height
        col = init_template.shape[1]  #width
        check = 255
        for row in range(rows): #720
            try:
                templist = init_template[row].tolist()
                firstval = templist.index(check)
                secondval = len(templist) - 1 - templist[::-1].index(check)
                myTemplate[row][firstval:secondval] = 255
            except ValueError:
                print('value is not in list..')
        #.............
        myTemplate = np.array(cv2.merge((zero_channel,myTemplate,zero_channel)),np.uint8)
        output = cv2.addWeighted(output, 1, myTemplate, 0.5, 0.0) #save green poly fill inside sliding windows
        cv2.imwrite('./test_images/output_conv_'+fname[12:], output)       
        #conv_warped = cv2.addWeighted(warpage, 1, myTemplate, 0.4, 0.0) # overlay the orignal road image with window results
        #end code to fill area between the lanes
    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

    # Display the final results
    #plt.imshow(output)
    #plt.title('window fitting results')
    #plt.show()
    cv2.imwrite('./test_images/conv_'+fname[12:], output)
        
    # assign warped to conv_warped, so that the area between the lanes is highlighted
    #warped # single dimension
    #conv_warped three dimensions
    #create the polyfit
    # this has to be understood and updated
    warp_hgt = warped.shape[0]
    yvals = range(0, warp_hgt)
    ploty = yvals
    res_yvals = np.arange(warp_hgt - (window_height/2), 0, -window_height)  # can also use linspace which also returns an array
        
    ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(res_yvals, leftx, 2)  # lefty, leftx
    right_fit = np.polyfit(res_yvals, rightx, 2)  # righty, rightx

    #left_fit = np.polyfit(lefty, leftx, 2)
    #right_fit = np.polyfit(righty, rightx, 2)
    
    try:
        left_fitx = left_fit[0] * np.power(ploty, 2) + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * np.power(ploty, 2) + right_fit[1] * ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1 * np.power(ploty, 2) + 1 * ploty
        right_fitx = 1 * np.power(ploty, 2) + 1 * ploty
        
    left_fitx = np.array(left_fitx, np.int32) 
    right_fitx = np.array(right_fitx, np.int32) 
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2, left_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
    middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2, right_fitx[::-1]+window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)

    
    out_img = np.dstack((warped, warped, warped)) * 255  #image of lines  # same as warpage
    #create red and blue lane lines
    road = np.zeros_like(out_img)  # this is a 3d warped image
    cv2.fillPoly(road, np.int_([left_lane]), (255, 0, 0))
    cv2.fillPoly(road, np.int_([right_lane]), (0, 0, 255))
    #print('rpoad: ', road)
    #make the lane lines white to create background for red and blue lane lines
    road_bkg = np.zeros_like(out_img)  # actual image
    cv2.fillPoly(road_bkg, [left_lane], (255, 255, 255))
    cv2.fillPoly(road_bkg, [right_lane], (255, 255, 255))

    road_warped = cv2.warpPerspective(road, Minv, img_size, flags=cv2.INTER_LINEAR)
    road_warped_bkg = cv2.warpPerspective(road_bkg, Minv, img_size, flags=cv2.INTER_LINEAR)

    #--------------
    #to add green poly fill
    lane_warped = cv2.warpPerspective(myTemplate, Minv, img_size, flags=cv2.INTER_LINEAR) #warp lane
    final_result = cv2.addWeighted(img_3d, 1, lane_warped, 0.4, 0.0) # overlay the orignal road image with window results
    img_3d = final_result
    #--------------
    base = cv2.addWeighted(img_3d, 1.0, road_warped_bkg, -1.0,0.0)
    result = cv2.addWeighted(base, 1.0, road_warped, 1.0,0.0)
    
     #cv2.fillPoly(result, np.int_([src]), (53, 255, 51))
    init_box = cv2.warpPerspective(road, Minv, img_size, flags=cv2.INTER_LINEAR)
    #cv2.imwrite('./test_images/finalZZZ_'+fname[12:], result)
    cv2.imwrite('./test_images/polyfill_'+fname[12:], result)
    
    #-------------------------------------------------------------------------------------
    # text on the image and video

    ploty = res_yvals
    ploty = np.array(res_yvals, np.int32)
    leftx_f = np.array(leftx, np.int32)
    rightx_f = np.array(rightx, np.int32)
    y_eval = np.max(ploty)
    
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx_f*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx_f*xm_per_pix, 2)

    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    #calculate offset of the car on the road
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_diff = (camera_center - warped.shape[1]/2) * xm_per_pix
    vehicle_pos = 'left'
    if center_diff <= 0:
        vehicle_pos = 'right'

    cv2.putText(result, 'Radius of Curvature = '+ str(round(right_curverad, 3))+'(m)', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    cv2.putText(result, 'Vehicle is '+str(abs(round(center_diff, 3)))+'(m) '+vehicle_pos+' of center', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    cv2.imwrite('./test_images/final_'+fname[12:], result)

    #an inefficient polyfill
    #cv2.fillPoly(result, np.int_([src]), (53, 255, 51))
    #cv2.imwrite('./test_images/finalZZZ_'+fname[12:], result)
    #plt.imshow(result)
    #plt.interactive(False)
    #plt.show()
    
    #----------------------------------------------------------
    # Colors in the left and right lane regions
    #out_img[lefty, leftx] = [255, 0, 0]  # color left lane pixels -- this info is missing in convolution, get from window code
    #out_img[righty, rightx] = [0, 0, 255] # color right lane pixels
    
    '''
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the 5 binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(mag_binary), mag_binary, dir_binary, rgb_r_binary, hls_h_binary, hls_s_binary)) * 255

    '''